## Análisis de datos extremos, ya sean pequeños o grandes (outliers)

Se exploran dos maneras de seleccionarlos:
* Un criterio para una sola variable en la manera más clásica: definir dos umbrales
* Una aproximación multidimensional

In [18]:
import pandas as pd
import numpy as np

n = 10**5
df = pd.read_csv(r"C:\COVID_pruebas\base_datos_2008.csv", nrows=n )

In [19]:
x = df["ArrDelay"].dropna()

### *Método I:*  definir dos umbrales

In [29]:
Q1 = np.percentile(x,25)
Q3 = np.percentile(x,75)
rangointer = Q3 - Q1

#Rango intercuantílico := Q3 - Q1

In [30]:
umbralinferior = Q1 - 1.5*rangointer
umbralsuperior = Q3 + 1.5*rangointer

print("Se consideran outliers aquellos que estén fuera de:",umbralinferior," y ",umbralsuperior)

Se consideran outliers aquellos que estén fuera de: -37.5  y  38.5


In [37]:
print("Numero de casos por 'encima' del umbral superior:", round( np.mean(x>38.5),4) )
print("Numero de casos por 'debajo' del umbral inferior:", round( np.mean(x<-37.5),4) )

print("\nDecimos entonces que la distribución de ArrDelay no es simétrica")

Numero de casos por 'encima' del umbral superior: 0.0839
Numero de casos por 'debajo' del umbral inferior: 0.0019

Decimos entonces que la distribución de ArrDelay no es simétrica


También es posible analizar varias variables a la vez y se hace con el paquete **sklearn.covariance**

In [38]:
from sklearn.covariance import EllipticEnvelope

outliers = EllipticEnvelope( contamination = 0.01 )
#Se crea un modelo que selecciona el 1% de los datos que considere que están muy alejados de nuestros datos más centrados.

In [43]:
var_list = ["DepDelay","TaxiIn","TaxiOut","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"]
#Se analiza toda una lista de variables, TODAS ELLAS SON CUANTITATIVAS.
# Este modelo asume la distribución normal de toda la lista de variables. ESTO NO TIENE POR QUÉ OCURRIR

In [47]:
x = np.array( df.loc[ :, var_list ].dropna() )
#Nuevamente se define la x

In [48]:
outliers.fit(x)
#Se entrena el modelo.
# "outliers", que es el modelo + .fit
# "x" son nuestros datos

EllipticEnvelope(contamination=0.01)

In [55]:
pred = outliers.predict(x)
#Pedimos que prediga con ese modelo, cuáles son los valores que forman parte del 1% que se buscan detectar.

pred 
#Esto es un array de 1 y -1. Los casos a detectar son los -1

elips_outliers = np.where( pred==-1 )[0]
#Esta es una lista de todos los "index" de aquellos csos que están muy alejados de los 
# valores centrales de todas las variables estudiadas, es decir, los "outliers"

elips_outliers

array([  444,  1161,  1188,  1191,  1412,  1418,  1503,  1634,  1911,
        1914,  2005,  2082,  2120,  2123,  2145,  2539,  2612,  2613,
        2725,  2771,  2937,  3120,  3589,  4146,  4265,  4430,  4437,
        4488,  4512,  4621,  4676,  4725,  5772,  5996,  5997,  6329,
        6358,  6515,  7245,  7257,  7460,  7480,  7481,  7620,  7698,
        7701,  7702,  7703,  7707,  7715,  8098,  8130,  8267,  8294,
        8298,  8327,  8484,  8566,  8683,  8691,  8865,  8993,  9922,
       10216, 10250, 10280, 10469, 10618, 10681, 10720, 10910, 11021,
       11231, 11237, 11242, 11316, 11955, 12085, 12088, 12240, 12452,
       12513, 12607, 12700, 12905, 12927, 12934, 13275, 13276, 13278,
       14259, 14282, 14285, 14297, 14318, 14358, 14367, 14510, 14544,
       14557, 14558, 14647, 14658, 14785, 14789, 14792, 14831, 14858,
       14870, 14871, 14880, 14881, 14940, 14945, 14946, 14947, 14948,
       14988, 15008, 15009, 15029, 15086, 15150, 15192, 15195, 15196,
       15197, 15202,

## Principios de las bases de datos relacionales

Se explora una de las estructuras más famosas para gestionar y almacenar bases de datos. El más utilizado cuando se trata de bases de datos ya planificadas.

* Las tablas se conectan con una **clave primaria**, que deben ser **únicas** y permitan identificar solo a los **registros** de cada tabla. 
* Las **claves primarias** no necesariamente deben ser únicas **en una sola tabla**. 
* En cada tabla, las combinaciones de **claves primarias**, debe de ser **única**.
* Mantiene la integridad referencial de los datos. No importa si se elimina un cliente, la estructura se mantiene.
* Cuando se tienen volúmenes importantes de datos, un reto es: **¿Qué estructura relacional se construirá para albergar la información?**

Diseñar una estructura de base de datos relacional es muy **complejo**, pero muy **eficiente**, pues hay que considerar principios de no duplicidad y la integridad de datos de manera correcta.

## Transformar un dataframe en una base de datos relacional

Se muestra un ejemplo práctico de cómo transformar un *dataframe* en una *base de datos relacional*

In [56]:
import pandas as pd

data = [(1, "Joan", "Gasull", 25, 1, "Libreta", 1.2, .4, 0.8, 3, "03-02-2018"),
       (1, "Joan", "Gasull", 25, 2, "Bolígrafo", 0.4, 0.15, 0.25, 1, "03-02-2018"),
       (1, "Joan", "Gasull", 25, 1, "Libreta", 1.2, .4, 0.8, 2, "15-02-2018"),
       (2, "Joan", "López", 33, 2, "Bolígrafo", 0.4, 0.15, 0.25, 4, "01-02-2018"),
       (2, "Joan", "López", 33, 1, "Libreta", 1.2, 0.4, 0.8, 10, "05-03-2018"),
       (3, "María", "García", 40, 1, "Libreta", 1.2, .4, 0.8, 20, "13-04-2018"),
       (3, "María", "García", 40, 2, "Bolígrafo", 0.4, 0.15, 0.25, 1, "09-02-2018"),
       (3, "María", "García", 40, 2, "Bolígrafo", 0.4, 0.15, 0.25, 3, "03-04-2018")]

labels = ["COMPRADOR_ID","NOMBRE","APELLIDO","EDAD","PRODUCTO_ID","PRODUCTO",
          "PRECIO","COSTE","MARGEN","CANTIDAD","FECHA"]

In [63]:
df = pd.DataFrame.from_records( data, columns=labels )
#Con esto se transforma la "data" a un "dataframe"

df.head(3)

,COMPRADOR_ID,NOMBRE,APELLIDO,EDAD,PRODUCTO_ID,PRODUCTO,PRECIO,COSTE,MARGEN,CANTIDAD,FECHA
0,1,Joan,Gasull,25,1,Libreta,1.2,0.40,0.80,3,03-02-2018
1,1,Joan,Gasull,25,2,Bolígrafo,0.4,0.15,0.25,1,03-02-2018
2,1,Joan,Gasull,25,1,Libreta,1.2,0.40,0.80,2,15-02-2018


Como se puede apreciar, hay nombres repetidos. Procedemos a dividir esta información en subtablas que contengan información necesaria.

Adicionalmente, es necesario mencionar que en éste ejemplo se tienen pocas columnas; sin embargo, si se tuvieran bastantes **campos/variables**, podría ser muy tedioso y complicado

In [76]:
compradores = df.drop_duplicates( subset=["COMPRADOR_ID","NOMBRE","APELLIDO","EDAD"], keep="first" )
compradores_2 = compradores.loc[ :,["COMPRADOR_ID","NOMBRE","APELLIDO","EDAD"] ]

compradores_2

,COMPRADOR_ID,NOMBRE,APELLIDO,EDAD
0,1,Joan,Gasull,25
3,2,Joan,López,33
5,3,María,García,40


In [77]:
productos = df.drop_duplicates( subset=["PRODUCTO_ID","PRODUCTO","PRECIO","COSTE","MARGEN"], keep="first" )
productos_2 = productos.loc[ :,["PRODUCTO_ID","PRODUCTO","PRECIO","COSTE","MARGEN"] ]

productos_2

,PRODUCTO_ID,PRODUCTO,PRECIO,COSTE,MARGEN
0,1,Libreta,1.2,0.40,0.80
1,2,Bolígrafo,0.4,0.15,0.25


In [78]:
compras = df.drop_duplicates( subset=["COMPRADOR_ID","PRODUCTO_ID","FECHA","CANTIDAD"], keep="first" )
compras_2 = compras.loc[ :,["COMPRADOR_ID","PRODUCTO_ID","FECHA","CANTIDAD"] ]
compras_2

,COMPRADOR_ID,PRODUCTO_ID,FECHA,CANTIDAD
0,1,1,03-02-2018,3
1,1,2,03-02-2018,1
2,1,1,15-02-2018,2
3,2,2,01-02-2018,4
4,2,1,05-03-2018,10
5,3,1,13-04-2018,20
6,3,2,09-02-2018,1
7,3,2,03-04-2018,3


## Joins. Trabajar con bases de datos relacionales.

Vamos a ver cómo operar con las principales funciones para **unir distintas bases de datos en una misma estructura**. Estas funciones se llaman **"joins"** y hay de distintos tipos.

In [80]:
import pandas as pd

consumidores = [("A","Móvil"),("B","Móvil"),("A","Portátil"),("A","Tablet"),
                ("B","Tablet"),("C","Portátil"),("D","Smartwatch"),("E","Consola")]

con_labels = ["Consumidor","Producto"]

con_df = pd.DataFrame.from_records( consumidores, columns=con_labels )

productores = [("a","Móvil"),("a","Smartwatch"),("a","Tablet"),("b","Portátil"),
               ("c","Sobremesa"),("c","Portátil")]

prod_labels = ["Productor","Producto"]

prod_df = pd.DataFrame.from_records( productores, columns=prod_labels )

In [86]:
con_df.head()

,Consumidor,Producto
0,A,Móvil
1,B,Móvil
2,A,Portátil
3,A,Tablet
4,B,Tablet


In [84]:
prod_df.head()
#Los resultados no están en orden bajo ningún criterio

,Productor,Producto
0,a,Móvil
1,a,Smartwatch
2,a,Tablet
3,b,Portátil
4,c,Sobremesa


In [98]:
unida = pd.merge(con_df,prod_df, on="Producto", how="outer")
#"merge" permite unir las dos bases de datos.
# "on" es la columna que aparece en ambas bases
# "how" es el tipo de join que se utilizará. En este caso, se recurré a "outer"

unida.tail()
#No hay consumidor de Sobremensa y tanpoco hay productor para las consolas

,Consumidor,Producto,Productor
6,A,Tablet,a
7,B,Tablet,a
8,D,Smartwatch,a
9,E,Consola,NaN
10,NaN,Sobremesa,c


In [101]:
nombres = list(unida.columns)
nombres

['Consumidor', 'Producto', 'Productor']

In [105]:
unida_2 = unida.dropna(subset=nombres)

unida_2.tail()
#Ya no hay vacíos

,Consumidor,Producto,Productor
4,C,Portátil,b
5,C,Portátil,c
6,A,Tablet,a
7,B,Tablet,a
8,D,Smartwatch,a


In [109]:
unida_3 = pd.merge(con_df,prod_df,how="inner")

unida_3
#Aquí aparece la intersección de los productos. No hay faltantes

,Consumidor,Producto,Productor
0,A,Móvil,a
1,B,Móvil,a
2,A,Portátil,b
3,A,Portátil,c
4,C,Portátil,b
5,C,Portátil,c
6,A,Tablet,a
7,B,Tablet,a
8,D,Smartwatch,a


In [117]:
pd.merge(con_df,prod_df,on="Producto", how="right")
#Nos une los dos data frames, pero SOLAMENTE SI encuentra la clave en el data frame de la derecha.
# Sucedería algo análogo a si se colocara how="left"

,Consumidor,Producto,Productor
0,A,Móvil,a
1,B,Móvil,a
2,D,Smartwatch,a
3,A,Tablet,a
4,B,Tablet,a
5,A,Portátil,b
6,C,Portátil,b
7,NaN,Sobremesa,c
8,A,Portátil,c
9,C,Portátil,c


## Paralelizar loops en Python 

Por **filtrado**, se entiende la **extracción de información relevante** en filas o columnas de datos.